## Feataure mapping on Image using OpenCV

In [5]:
import cv2
import numpy as np
from tkinter import Tk, filedialog,StringVar, Button, Label, messagebox, IntVar, OptionMenu, Scale, HORIZONTAL
from PIL import Image, ImageTk  # Pillow for image display in Tkinter

In [8]:


def show_image_on_label(img_cv, label_widget, maxsize=(250, 250)):
    if img_cv is None:
        label_widget.config(image='', text='No Image')
        return
    if len(img_cv.shape) == 2:  # Grayscale
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_GRAY2RGB)
    else:  # Color
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_cv)
    img_pil.thumbnail(maxsize)
    img_tk = ImageTk.PhotoImage(img_pil)
    label_widget.img_tk = img_tk  # Prevent garbage collection
    label_widget.config(image=img_tk, text='')

def select_image(label, is_first=True):
    global img1, img2, file_path1, file_path2
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    try:
        mode = cv2.IMREAD_COLOR if color_var.get() == "Color" else cv2.IMREAD_GRAYSCALE
        img = cv2.imread(file_path, mode)
        if img is None:
            raise ValueError("File is not a valid image.")
        if is_first:
            img1 = img
            file_path1 = file_path
        else:
            img2 = img
            file_path2 = file_path
        label.config(text=f"Image: {file_path.split('/')[-1]}")
        show_image_on_label(img, label)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load image: {e}")

def resize_cv_image(img, max_width=900, max_height=700):
    h, w = img.shape[:2]
    scale = min(max_width / w, max_height / h, 1.0)
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def feature_matching():
    if img1 is None or img2 is None:
        messagebox.showwarning("Input Error", "Please select both images first.")
        return

    detector_type = detector_var.get()
    nfeatures = orb_features_var.get()
    try:
        if detector_type == 'ORB':
            detector = cv2.ORB_create(nfeatures=nfeatures)
            norm_type = cv2.NORM_HAMMING
        elif detector_type == 'SIFT':
            detector = cv2.SIFT_create(nfeatures=nfeatures)
            norm_type = cv2.NORM_L2
        else:
            raise ValueError("Unknown detector selected.")

        keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
        keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
        if descriptors1 is None or descriptors2 is None:
            raise ValueError("No features found in one or both images.")

        bf = cv2.BFMatcher(norm_type, crossCheck=True)
        matches = bf.match(descriptors1, descriptors2)
        if not matches:
            raise ValueError("No matches found between images.")

        matches = sorted(matches, key=lambda x: x.distance)
        img_matches = cv2.drawMatches(
            img1, keypoints1, img2, keypoints2, matches[:50], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        img_matches = resize_cv_image(img_matches)
        cv2.imshow('Feature Matching', img_matches)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except Exception as e:
        messagebox.showerror("Feature Matching Error", f"Failed to match features: {e}")

root = Tk()
root.title('Feature Matching')

img1 = img2 = None
file_path1 = file_path2 = None

# Detector selection
detector_options = ['ORB', 'SIFT']
detector_var = StringVar(root)
detector_var.set('ORB')
Label(root, text="Feature Detector:").pack()
OptionMenu(root, detector_var, *detector_options).pack()

# ORB/SIFT nfeatures parameter
Label(root, text="Number of Features:").pack()
orb_features_var = IntVar(value=500)
Scale(root, from_=100, to=2000, orient=HORIZONTAL, variable=orb_features_var).pack()

# Color or Grayscale
color_var = StringVar(root)
color_var.set("Grayscale")
Label(root, text="Image Mode:").pack()
OptionMenu(root, color_var, "Grayscale", "Color").pack()

# Image 1 selection and display
label_img1 = Label(root, text="Image 1: Not selected")
label_img1.pack()
btn_select_image1 = Button(root, text="Select Image 1", command=lambda: select_image(label_img1, is_first=True))
btn_select_image1.pack()

# Image 2 selection and display
label_img2 = Label(root, text="Image 2: Not selected")
label_img2.pack()
btn_select_image2 = Button(root, text="Select Image 2", command=lambda: select_image(label_img2, is_first=False))
btn_select_image2.pack()

# Match features button
btn_match_features = Button(root, text="Match Features", command=feature_matching)
btn_match_features.pack()

root.mainloop()
